In [28]:
# %matplotlib inline
import pandas  as pd  
import numpy as np
import re
import math
import os; 
  
from health.utils import list_files,slugify, DTYPE_DICT, STR_COLS,NAME_MAP , EXEC_MODE

%reload_ext autoreload
%autoreload 2

TARGET='Depression'


[autoreload of health.utils failed: Traceback (most recent call last):
  File "/Users/ceegees/workspace/iisc-learning/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/ceegees/workspace/iisc-learning/.venv/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/local/Cellar/python@3.12/3.12.6/Frameworks/Python.framework/Versions/3.12/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/Users/ceegees/workspace/iisc-learning/are-you-depressed/health/utils.py", line 20, in <module>
    model = load(f"../models/logistic-{EXEC_MODE}.job

Cleans up the columns from raw data to generate processed data
Iterate through all folders in the raw folder and generates corresponding cleaned files in the

Actions done
1. Cleans up CGPA to buckets
2. Cleans up Sleep Duration
3. Cleans up values for City, Degree,Dietary Habits, Profession

In [29]:

def cleanup_CGPA(val):
    # if isinstance(float(val),float):
    #     return math.floor(float(val)*10)/10
    num_val = float(val)
    if math.isnan(num_val) or val == "":
        return -1
    return math.floor(float(num_val) * 10) / 10


Cleans up random sleep values to a number and default to 6 hours of sleep

In [30]:

two_digit_pattern = r"(\d+){1}\s*[^\d]{1}+(\d+)"
single_number_pattern = r".*([\d]{2})"

def cleanup_sleep(sleep):
    if sleep == "More than 8 hours":
        return 8
    elif sleep == "Less than 5 hours":
        return 4.5
    elif sleep == "Moderate":
        return 6
    elif re.match(two_digit_pattern, sleep):
        matches = re.findall(two_digit_pattern, sleep)
        num1, num2 = matches[0]
        # return sleep
        total = (int(num1) + int(num2)) / 2
        if total < 10:
            return total
        return 6

    else:
        # print('Sleep non confirmant value', sleep)
        # raise Exception(f'Invalid value for sleep duration {sleep}')
        return 6


Cleanup with an allowed list of values to remove error data ,everything not found is bucketed to other or a default value

In [31]:


def cleanup_with_lables(name, allowed=[]):
    if name == '':
        return '';
    if name in allowed:
        return name 
    return allowed[-1]
     


In [32]:
## Check unique labels on each cols
raw_files = list_files(f"../data/raw/{EXEC_MODE}/")

df_list = []
for f in raw_files:
    if not ("train" in f):
        continue

    d = pd.read_csv(
        f, dtype= DTYPE_DICT
    )
    df_list.append(d)
            # Combine the list of dataframes
df =  pd.concat(df_list)
cols = df.columns.to_list() 
cols.remove("id")
cols.remove("Name")
print(f"Analyising {len(df)}")


for col in cols:
    df2 = df[col].value_counts().to_frame('Count').reset_index()
    # print(f"Generated {col}")
    max_val = df2['Count'].max()
    row_len = len (df2)
    # print(col , row_len,max_val)
    if (col in STR_COLS)  : 
        print(f'Cleaning up {col} - {row_len} {df2.columns.to_list()} - max {max_val}' )
        df3 = df2[df2['Count'] >= max_val/100 ] 
        other_count = df2[df2['Count'] < max_val/100 ]['Count'].sum()
        if other_count > 0:
            df3 = df3.rename(columns={col: 'name'})
            df3 = df3._append({'name':'Other','Count':other_count },ignore_index=True)
            print(f"{col} - total : {other_count}")
            print(df2[df2['Count'] < max_val/100] )

        df3.to_csv(f'../reports/raw/labels-{slugify(col)}.csv',index=False)
    df2.to_csv(f'../reports/raw/vals_{slugify(col)}.csv',index=False)



Analyising 140700
Cleaning up Gender - 2 ['Gender', 'Count'] - max 77464
Cleaning up City - 98 ['City', 'Count'] - max 6591
City - total : 98
          City  Count
30       Mihir      7
31     Nandini      4
32      Harsha      3
33    Pratyush      3
34        City      3
..         ...    ...
93    Malyansh      1
94      M.Tech      1
95  Less Delhi      1
96       Plata      1
97      Chhavi      1

[68 rows x 2 columns]
Cleaning up Profession - 64 ['Profession', 'Count'] - max 24906
Profession - total : 50
              Profession  Count
35               Student      7
36              Academic      5
37                   BCA      3
38                Yogesh      3
39            Unemployed      3
40            Profession      3
41                   PhD      2
42                   MBA      2
43                   LLM      2
44               Analyst      1
45                   BBA      1
46                 Patna      1
47                  MBBS      1
48                  M.Ed      1
49 

In [33]:
def cleanup_diet(val) :

    if val == 'Unhealthy':
        return 0
    elif val == 'Healthy':
        return 1
    else:
        return 0.5

def cleanup(val, col, labels=[]):

    if col == "CGPA":
        return cleanup_CGPA(val)
    elif col == "Sleep Duration":
        return cleanup_sleep(val)
    elif col == "Age":
        return math.floor(val)
    # elif col == ''
    elif EXEC_MODE == 'v2' and (col == 'Have you ever had suicidal thoughts ?'  or col == 'Family History of Mental Illness'):
        return 1 if val == 'Yes' else 0
    elif EXEC_MODE == 'v2' and col == 'Dietary Habits':
        return cleanup_diet(val)
    elif EXEC_MODE =="v2" and col == "Working Professional or Student":
        return  0 if val == 'Student' else 1
    elif len(labels) > 0:
        return cleanup_with_lables(val, labels)
    else:
        return val
 

def clean_data(input, column_names):
    result = pd.DataFrame()
    result["id"] = input["id"]
    for col in column_names:
        try:
            sl = slugify(col)
            labels = []
            label_file = f"../health/labels/{sl}.csv"
            if os.path.isfile(label_file):
                df = pd.read_csv(filepath_or_buffer=label_file) 
                labels = df['name'].unique()
                print(f"reading {label_file} : ",len(labels))
            result[col] = input[col].apply(lambda x: cleanup(x, col, labels=labels))
            # break
        except Exception as err:
            print("error occured", err)
    return result

def clean_all_files():
    raw_files = list_files(f"../data/raw/{EXEC_MODE}")
     
    for f in raw_files:
        if not (".csv" in f):
            continue

        target = f.replace("/raw/", f"/processed/")
        if (os.path.isfile(target)):
            print(f"File '{target}' already exists")
            continue;
        print(f"processing '{f}' ")
        df = pd.read_csv(
            f, dtype= DTYPE_DICT
        )
        column_names = df.keys().to_list()
        column_names.remove("id")
        column_names.remove("Name")
        # result = clean_data(df, column_names) 

        result = clean_data(df, column_names) 
        result = result.rename(columns=NAME_MAP)
        result.to_csv(target,index=False)
        print(f"Completed '{f}' ") 

clean_all_files()

processing '/Users/ceegees/workspace/iisc-learning/are-you-depressed/data/raw/v1/test.csv' 
reading ../health/labels/city.csv :  31
reading ../health/labels/profession.csv :  37
reading ../health/labels/dietary-habits.csv :  3
reading ../health/labels/degree.csv :  28
Completed '/Users/ceegees/workspace/iisc-learning/are-you-depressed/data/raw/v1/test.csv' 
processing '/Users/ceegees/workspace/iisc-learning/are-you-depressed/data/raw/v1/train-5.csv' 
reading ../health/labels/city.csv :  31
reading ../health/labels/profession.csv :  37
reading ../health/labels/dietary-habits.csv :  3
reading ../health/labels/degree.csv :  28
Completed '/Users/ceegees/workspace/iisc-learning/are-you-depressed/data/raw/v1/train-5.csv' 
processing '/Users/ceegees/workspace/iisc-learning/are-you-depressed/data/raw/v1/train-4.csv' 
reading ../health/labels/city.csv :  31
reading ../health/labels/profession.csv :  37
reading ../health/labels/dietary-habits.csv :  3
reading ../health/labels/degree.csv :  28
Co